# Projeto Predição de Filme para o Usuário

In [93]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

df = pd.read_csv('ds_netflix/n_movies.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [94]:
# Primeiras 10 linhas do DataSet
df.head(10)

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"
5,The Sandman,(2022– ),TV-MA,45 min,"Drama, Fantasy, Horror",7.8,Upon escaping after decades of imprisonment by...,"['Tom Sturridge, ', 'Boyd Holbrook, ', 'Patton...","116,358"
6,Rick and Morty,(2013– ),TV-MA,23 min,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"['Justin Roiland, ', 'Chris Parnell, ', 'Spenc...","502,160"
7,Breaking Bad,(2008–2013),TV-MA,49 min,"Crime, Drama, Thriller",9.5,A high school chemistry teacher diagnosed with...,"['Bryan Cranston, ', 'Aaron Paul, ', 'Anna Gun...","1,831,340"
8,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,After an experimental gene therapy turns them ...,"['Morgan Taylor Campbell, ', 'Italia Ricci, ',...","3,123"
9,Blonde,(2022),NC-17,166 min,"Biography, Drama, Mystery",6.2,A fictionalized chronicle of the inner life of...,"['Andrew Dominik', '| ', ' Stars:', 'Ana de...",935


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9957 entries, 0 to 9956
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9957 non-null   object 
 1   year         9430 non-null   object 
 2   certificate  6504 non-null   object 
 3   duration     7921 non-null   object 
 4   genre        9884 non-null   object 
 5   rating       8784 non-null   float64
 6   description  9957 non-null   object 
 7   stars        9957 non-null   object 
 8   votes        8784 non-null   object 
dtypes: float64(1), object(8)
memory usage: 700.2+ KB


## TfidfVectorizer:

- Converte as sinopses (description) em vetores numéricos usando a técnica TF-IDF.

- TF-IDF calcula a importância de cada palavra no contexto do documento (ex.: "supernatural" tem peso alto em "Stranger Things").

- stop_words='english': Remove palavras comuns (como "the", "and").

## tfidf_matrix:

- Matriz onde cada linha representa um filme e cada coluna uma palavra do vocabulário.

- Os valores são os pesos TF-IDF (ex.: [0.2, 0.5, 0.0, ...]).

## linear_kernel:

- Calcula a similaridade de cosseno entre todos os filmes.

- Compara os vetores TF-IDF para medir quão similares são as sinopses.

- Saída: Matriz de similaridade (cosine_sim), onde cosine_sim[i][j] é a similaridade entre o filme i e j (valor entre 0 e 1).

In [96]:
# 2. Pré-processamento e modelo
# Remove palavras comuns (como "the", "and", "is") que não agregam significado.
tfidf = TfidfVectorizer(stop_words='english')
# Preencher valores ausentes na coluna 'description' com uma string vazia
tfidf_matrix = tfidf.fit_transform(df['description'])
# Calcular a matriz de similaridade do cosseno
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

## Busca pelo filme digitado:

- Converte o título para minúsculas e procura no DataFrame.

- Se não encontrar, retorna None (erro tratado depois).

## Cálculo das similaridades:

- enumerate(cosine_sim[idx]): Pega a linha da matriz cosine_sim correspondente ao filme escolhido e enumera os índices.

- Exemplo: [(0, 0.1), (1, 0.9), ...] (índice do filme, similaridade).

## Ordenação e seleção:

- Ordena os filmes por similaridade (do maior para o menor).

- Pega os 3 mais similares (excluindo o próprio filme, por isso [1:4]).

- Retorno das recomendações:

- Extrai os índices dos filmes similares e retorna seus títulos.

In [97]:
# 3. Função de recomendação com tratamento de erro
# Função para recomendar filmes com base no título
def recommend_movies(title, cosine_sim=cosine_sim, df=df):
    try:
        idx = df[df['title'].str.lower() == title.lower()].index[0]
    except IndexError:
        return None  # Filme não encontrado
    
    # Obter os índices dos filmes mais similares
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Ordenar os filmes com base na similaridade
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Obter os títulos dos filmes mais similares
    sim_scores = sim_scores[1:4]  # Top 3 (exclui o próprio filme)
    # Extrair os índices dos filmes
    movie_indices = [i[0] for i in sim_scores]
    # Retornar os títulos dos filmes recomendados
    return df['title'].iloc[movie_indices]

## Loop infinito:

- O programa roda até o usuário digitar sair.

## Input do usuário:

- Pede o nome de um filme/série.

## Tratamento de erros:

- Se recommend_movies retornar None, mostra mensagem de erro e lista os filmes disponíveis.

## Saída das recomendações:

- Se o filme for válido, imprime os 3 mais similares.

In [98]:
# 4. Input do usuário mais o loop
# Aqui o usuário pode inserir o nome de um filme ou série
while True:
    user_input = input("\nDigite o nome de um filme/série (ou 'sair' para encerrar): ")
    
    # Verifica se o usuário deseja sair
    if user_input.lower() == 'sair':
        print("Até logo meu queridx! :)")
        break
    
    # Chama a função de recomendação
    recommendations = recommend_movies(user_input)
    
    # Exibe as recomendações ou uma mensagem de erro
    if recommendations is None:
        print("Filme não encontrado. Tente novamente. Opções válidas:")
        print(df['title'].to_string(index=False))
    else:
        print("\nRecomendações:")
        print(recommendations.to_string(index=False))


Recomendações:
   Red Dot
The Chalet
  Bird Box

Recomendações:
   Red Dot
The Chalet
  Bird Box

Recomendações:
Dark Summer
       Prey
   Marianne

Recomendações:
         Bird Box
   Vaastu Shastra
The War Next-door
Filme não encontrado. Tente novamente. Opções válidas:
                                         Cobra Kai
                                         The Crown
                                  Better Call Saul
                                     Devil in Ohio
                            Cyberpunk: Edgerunners
                                       The Sandman
                                    Rick and Morty
                                      Breaking Bad
                                    The Imperfects
                                            Blonde
                                   Stranger Things
                                   End of the Road
                                  The Walking Dead
                 Glass Onion: A Knives Out Mystery
           